# Results extraction for paper

In [18]:
import os
import glob
import json
import shutil
import pandas as pd
import numpy as np

## general blurb results table

first check missing results in output dirs

In [2]:
for res_dir in os.listdir("out"):
    if "debug" in res_dir : continue
    pred_res_path = os.path.join('out',res_dir,'predict_results.json')
    if not os.path.exists(pred_res_path):
        print(res_dir, "misses results")
        shutil.rmtree(os.path.join('out',res_dir))

In [13]:
exp_results = []
for pred_path in glob.glob("out/*/predict_results.json"):
    if "debug" in pred_path: continue
    metrics_results = json.load(open(pred_path))
    metrics_results = {k.replace("predict_",""):v for k,v in metrics_results.items()}
    if "ncbi_disease" in pred_path:
        pred_path = pred_path.replace("ncbi_disease","ncbi-disease")
    
    model, corpus, seed = pred_path.split('/')[-2].split('_')
    config = {"model":model,"corpus":corpus,"seed":int(seed.replace('seed',''))}
    exp_results.append(config|metrics_results)

In [39]:
df = pd.DataFrame(exp_results).sort_values(by=["model","corpus","seed"])
df = df[["model","corpus","f1"]]
df

,model,corpus,f1
19,bert-base-uncased,blurb-bc2gm,0.749758
9,bert-base-uncased,blurb-bc2gm,0.746618
12,bert-base-uncased,blurb-bc2gm,0.753172
5,bert-base-uncased,blurb-bc2gm,0.768449
23,bert-base-uncased,blurb-bc2gm,0.748085
14,bert-base-uncased,blurb-bc5chem,0.880308
17,bert-base-uncased,blurb-bc5chem,0.863208
0,bert-base-uncased,blurb-bc5chem,0.874118
2,bert-base-uncased,blurb-bc5chem,0.871561
7,bert-base-uncased,blurb-bc5chem,0.876540


In [40]:
# Group by model and corpus and calculate mean and std
str_mean_and_std = lambda x : "{:.2f}%+/-{:.2f}".format(np.mean(x)*100,np.std(x)*100)

In [41]:
agg_df = df.groupby(["model","corpus"]).agg(str_mean_and_std)
agg_df

f1
model             corpus                           
bert-base-uncased blurb-bc2gm         75.32%+/-0.79
                  blurb-bc5chem       87.31%+/-0.57
                  blurb-bc5disease    77.09%+/-1.01
                  blurb-jnlpba        76.77%+/-0.86
                  blurb-ncbi-disease  81.59%+/-1.03